<a href="https://colab.research.google.com/github/OlgaNezh/NLP/blob/main/colab_text_classification_part1_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O imdb.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vrQ5czMHoO3pEnmofFskymXMkq_u1dPc"
!unzip imdb.zip
!pip -q install eli5
!pip -q install spacy
!python -m spacy download en

Archive:  imdb.zip
  inflating: test.tsv                
  inflating: train.tsv               
     |████████████████████████████████| 216 kB 6.1 MB/s 
     |████████████████████████████████| 133 kB 58.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Классификация текстов

Начнём с самого простого - анализа тональности текста.

Будем классифицировать отзывы с IMDB на положительные/отрицательные.

Датасет взят с http://ai.stanford.edu/~amaas/data/sentiment/

In [2]:
!head train.tsv

is_positive	review
0	"Dreamgirls, despite its fistful of Tony wins in an incredibly weak year on Broadway, has never been what one would call a jewel in the crown of stage musicals. However, that is not to say that in the right cinematic hands it could not be fleshed out and polished into something worthwhile on-screen. Unfortunately, what transfers to the screen is basically a slavishly faithful version of the stage hit with all of its inherent weaknesses intact. First, the score has never been one of the strong points of this production and the film does not change that factor. There are lots of songs (perhaps too many?), but few of them are especially memorable. The closest any come to catchy tunes are the title song and One Night Only - the much acclaimed And I Am Telling You That I Am Not Going is less a great song than it is a dramatic set piece for the character of Effie (Jennifer Hudson). The film is slick and technically well-produced, but the story and characters are surprisi

In [34]:
#Начинаем классифицировать
positive_words = ['love', 'great', 'best', 'wonderful']
negative_words = ['worst', 'awful', '1/10', 'crap']

positives_count = test_df.review.apply(lambda text: sum(word in text for word in positive_words))
negatives_count = test_df.review.apply(lambda text: sum(word in text for word in negative_words))
is_positive = positives_count > negatives_count
correct_count = (is_positive == test_df.is_positive).values.sum()

accuracy = correct_count / len(test_df)

print('Test accuracy = {:.2%}'.format(accuracy))

Test accuracy = 66.73%


In [35]:
positive_words =  ['great','best', 'wonderful', 'breathtaking', 'first-rate', 'brilliant',
                   'magnetic', 'insightful', 'well-paced', 'must-see', 'charismatic', 
                   'enjoyable', 'hilarious', 'absorbing', 'riveting', 'intriguing', 
                   'powerful', 'fascinating', 'excellent',  'perfect', 'surprised',
                   'delightful', 'incredible', 'fantastic', 'ideal', 'amazing',
                   'refreshing', 'wonderfully', 'funniest', 'surprisingly',
                   'rare', 'superb']
negative_words =  ['worst', 'awful', '1/10', 'crap', 'second-rate',
                    'moronic', 'third-rate',  'juvenile','boring',
                  'worse', 'dull', 'mildly', 'redeeming', 'baldwin', 'weak', 'wooden']

positives_count = test_df.review.apply(lambda text: sum(word in text for word in positive_words))
negatives_count = test_df.review.apply(lambda text: sum(word in text for word in negative_words))
is_positive = positives_count > negatives_count
correct_count = (is_positive == test_df.is_positive).values.sum()

accuracy = correct_count / len(test_df)

print('Test accuracy = {:.2%}'.format(accuracy))

Test accuracy = 72.84%


In [36]:
import pandas as pd

train_df = pd.read_csv("train.tsv", delimiter="\t")
test_df = pd.read_csv("test.tsv", delimiter="\t")

print('Train size = {}'.format(len(train_df)))
print('Test size = {}'.format(len(test_df)))

Train size = 25000
Test size = 25000


Посмотрите глазами на тексты? Какие есть зацепки, как определить, что это за сентимент?

Самое простое, как всегда - найти ключевые слова.

**Задание** Придумайте хорошие ключевые слова или фразы и наберите хотя бы 71% точности на тесте (и не забудьте посмотреть на код классификации!)

**Задание** Кому-нибудь нравятся эти `<br /><br />`? Лично мне - нет. Напишите регулярку, которая будет их удалять

In [37]:
import re

pattern = re.compile('<br />')

print(train_df['review'].iloc[3])
print(pattern.subn(' ', train_df['review'].iloc[3])[0])

Spoilers ahead if you want to call them that...<br /><br />I would almost recommend this film just so people can truly see a 1/10. Where to begin, we'll start from the top...<br /><br />THE STORY: Don't believe the premise - the movie has nothing to do with abandoned cars, and people finially understanding what the mysterious happenings are. It's a draub, basic, go to cabin movie with no intensity or "effort".<br /><br />THE SCREENPLAY: I usually give credit to indie screenwriters, it's hard work when you are starting out...but this is crap. The story is flat - it leaves you emotionless the entire movie. The dialogue is extremely weak and predictable boasting lines of "Woah, you totally freaked me out" and "I was wondering if you'd uh...if you'd like to..uh, would you come to the cabin with me?". It makes me want to rip out all my hair, one strand at a time and feed it to myself.<br /><br />THE CHARACTERS: HOLY CRAP!!!! Some have described the characters as flat, I want to take it one 

Применим ее:

In [38]:
train_df['review'] = train_df['review'].apply(lambda text: pattern.subn(' ', text)[0])
test_df['review'] = test_df['review'].apply(lambda text: pattern.subn(' ', text)[0])

Пора переходить к машинке!

Как будем представлять текст? Проще всего - мешком слов.

Заведём большой-большой словарь - список всех слов в обучающей выборке. Тогда каждое предложение можно представить в виде вектора, в котором будет записано, сколько раз встретилось каждое из возможных слов:

![bow](https://raw.githubusercontent.com/DanAnastasyev/DeepNLP-Course/master/Week%2001/Images/BOW.png)

Простой и приятный способ сделать это - запихнуть тексты в `CountVectorizer`.

Он имеет такую сигнатуру:

```python
CountVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern=r'(?u)\b\w\w+\b', ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.int64'>)
```

Для начала обратим внимание на параметры `lowercase=True` и `max_df=1.0, min_df=1, max_features=None` - они про то, что по умолчанию все слова будут приводиться к нижнему регистру и в словарь попадут все слова, встречавшиеся в текстах.

При желании можно было бы убрать слишком редкие или слишком частотные слова - пока не будем этого делать.

Посмотрим на простом примере, как он будет работать:

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

dummy_data = ['The movie was excellent',
              'the movie was awful']

dummy_matrix = vectorizer.fit_transform(dummy_data)

print(dummy_matrix.toarray())
print(vectorizer.get_feature_names())

[[0 1 1 1 1]
 [1 0 1 1 1]]
['awful', 'excellent', 'movie', 'the', 'was']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


*Как именно vectorizer определяет границы слов? Обратите внимание на параметр `token_pattern=r'(?u)\b\w\w+\b'` - как он будет работать?*

Запустим его на реальных данных:

In [40]:
vectorizer = CountVectorizer()
vectorizer.fit(train_df['review'].values)

CountVectorizer()

Посмотрим на слова, попавшие в словарь:

In [41]:
vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '0000000000001',
 '00001',
 '00015',
 '000s',
 '001',
 '003830',
 '006',
 '007',
 '0079',
 '0080',
 '0083',
 '0093638',
 '00am',
 '00pm',
 '00s',
 '01',
 '01pm',
 '02',
 '020410',
 '029',
 '03',
 '04',
 '041',
 '05',
 '050',
 '06',
 '06th',
 '07',
 '08',
 '087',
 '089',
 '08th',
 '09',
 '0f',
 '0ne',
 '0r',
 '0s',
 '10',
 '100',
 '1000',
 '1000000',
 '10000000000000',
 '1000lb',
 '1000s',
 '1001',
 '100b',
 '100k',
 '100m',
 '100min',
 '100mph',
 '100s',
 '100th',
 '100x',
 '100yards',
 '101',
 '101st',
 '102',
 '102nd',
 '103',
 '104',
 '1040',
 '1040a',
 '1040s',
 '105',
 '1050',
 '105lbs',
 '106',
 '106min',
 '107',
 '108',
 '109',
 '10am',
 '10lines',
 '10mil',
 '10min',
 '10minutes',
 '10p',
 '10pm',
 '10s',
 '10star',
 '10th',
 '10x',
 '10yr',
 '11',
 '110',
 '1100',
 '11001001',
 '1100ad',
 '111',
 '112',
 '1138',
 '114',
 '1146',
 '115',
 '116',
 '117',
 '11f',
 '11m',
 '11th',
 '12',
 '120',
 '1200',
 '1200f',
 '1201',
 '1202',
 '123',
 '12383499143743701',
 '1

Попробуем кого-нибудь таки сконвертировать

In [42]:
vectorizer.transform([train_df['review'].iloc[3]])

<1x74849 sparse matrix of type '<class 'numpy.int64'>'
	with 206 stored elements in Compressed Sparse Row format>

То, что и хотели - вектор с bow (т.е. bag-of-words) представлением исходного текста.

И чем эта информация может помочь? Ну, всё тем же - какие-то слова носят положительный окрас, какие-то - отрицательный. Большинство вообще нейтральный, да.

![bow with weights](https://github.com/DanAnastasyev/DeepNLP-Course/raw/master/Week%2001/Images/BOW_weights.png)

Хочется, наверное, подобрать коэффициенты, которые будут определять уровень окраса, да? Подбирать нужно по обучающей выборке, а не как мы перед этим делали.

Например, для выборки
```
1   The movie was excellent
0   the movie was awful
```
легко подобрать коэффициенты на глазок: что-нибудь вроде `+1` для `excellent`,  `-1` для `awful` и по нулям всем остальным.

Построим линейную модель, которая станет этим заниматься. Она будет учиться строить разделяющую гиперплоскость в пространстве bow-векторов.

Проверим, как справится логистическая регрессия с нашей супер-выборкой из пары предложений

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

dummy_data = ['The movie was excellent',
              'the movie was awful']
dummy_labels = [1, 0]

vectorizer = CountVectorizer()
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(dummy_data, dummy_labels)

print(vectorizer.get_feature_names())
print(classifier.coef_)

['awful', 'excellent', 'movie', 'the', 'was']
[[-0.40104279  0.40104279  0.          0.          0.        ]]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Получилось что надо.

Запустим теперь её на реальных данных.

In [44]:
model.fit(train_df['review'], train_df['is_positive'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', LogisticRegression())])

In [45]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def eval_model(model, test_df_X, test_df_Y):
    preds = model.predict(test_df_X)
    print('Test precision = {:.2%}'.format(precision_score(test_df_Y, preds)))
    print('Test recall = {:.2%}'.format(recall_score(test_df_Y, preds)))
    print('Test f1 = {:.2%}'.format(f1_score(test_df_Y, preds)))
    print('Test accuracy = {:.2%}'.format(accuracy_score(test_df_Y, preds)))
    
eval_model(model, test_df['review'], test_df['is_positive'])

Test precision = 87.04%
Test recall = 85.77%
Test f1 = 86.40%
Test accuracy = 86.50%


Прогресс!

Хочется как-то посмотреть, что заинтересовало классификатор. К счастью, сделать это совсем просто:

In [46]:
import eli5
eli5.show_weights(classifier, vec=vectorizer, top=40)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weight?,Feature
+1.855,refreshing
+1.760,wonderfully
+1.689,funniest
+1.647,surprisingly
+1.626,rare
+1.432,superb
+1.401,excellent
+1.365,incredible
+1.351,perfect
+1.302,delightful


Посмотрим на конкретные примеры его работы:

In [47]:
print('Positive' if test_df['is_positive'].iloc[1] else 'Negative')
eli5.show_prediction(classifier, test_df['review'].iloc[1], vec=vectorizer, 
                     targets=['positive'], target_names=['negative', 'positive'])

Positive


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [48]:
print('Positive' if test_df['is_positive'].iloc[6] else 'Negative')
eli5.show_prediction(classifier, test_df['review'].iloc[6], vec=vectorizer, 
                     targets=['positive'], target_names=['negative', 'positive'])

Negative


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Посмотрим на примеры неправильной классификации, наконец:

In [49]:
import numpy as np

preds = model.predict(test_df['review'])
incorrect_pred_index = np.random.choice(np.where(preds != test_df['is_positive'])[0])

eli5.show_prediction(classifier, test_df['review'].iloc[incorrect_pred_index],
                     vec=vectorizer, targets=['positive'], target_names=['negative', 'positive'])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Придумываем новые признаки

### Tf-idf

Сейчас мы на все слова смотрим с одинаковым весом - хотя какие-то из них более редкие, какие-то более частые, и эта частотность - полезная, вообще говоря, информация.

Самый простой способ добавить статистическую информацию о частотностях - сделать *tf-idf* взвешивание:

$$\text{tf-idf}(t, d) = \text{tf}(t, d) \times \text{idf}(t)$$

*tf* - term-frequency - частотность слова `t` в конкретном документе `d` (рецензии в нашем случае). Это ровно то, что мы уже считали.

*idf* - inverse document-frequency - коэффициент, который тем больше, чем в меньшем числе документов встречалось данное слово. Считается как-нибудь так:
$$\text{idf}(t) = \text{log}\frac{1 + n_d}{1 + n_{d(t)}} + 1$$
где $n_d$ - число всех документов, а $n_{d(t)}$ - число документов со словом `t`.

Использовать его просто - нужно заменить `CountVectorizer` на `TfidfVectorizer`.

**Задание** Попробуйте запустить `TfidfVectorizer`. Посмотрите на ошибки, которые он научился исправлять, и на ошибки, которые он начал делать - по сравнению с `CountVectorizer`.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['review'], train_df['is_positive'])

eval_model(model, test_df)

Test accuracy = 88.28%


### N-граммы слов

До сих пор мы смотрели на тексты как на мешок слов - но очевидно, что есть разница между `good movie` и `not good movie`.

Добавим информацию (хоть какую-то) о последовательностях слов - будем извлекать еще и биграммы слов.

В Vectorizer'ах для этого есть параметр `ngram_range=(n_1, n_2)` - он говорит, что нужны n_1-...n_2-граммы.

**Задание** Попробуйте увеличенный range и поинтерпретируйте полученный результат.

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['review'], train_df['is_positive'])

eval_model(model, test_df['review'], test_df['is_positive'])

Test precision = 88.37%
Test recall = 88.17%
Test f1 = 88.27%
Test accuracy = 88.28%


### N-граммы символов

Символьные n-граммы дают простой способ выучить полезные корни и суффиксы, не связываясь с этой вашей лингвистикой - только статистика, только хардкор.

Например, слово `badass` мы можем представить в виде такой последовательности триграмм:

`##b #ba bad ada das ass ss# s##`

So interpretable, неправда ли?

Реализовать это дело всё так же просто - нужно поставить `analyzer='char'` в вашем любимом Vectorizer'е и выбрать размер `ngram_range`.

**Задание** Запилите классификатор на n-граммах символов и визуализируйте его.

In [52]:
vectorizer = TfidfVectorizer(ngram_range=(2, 6), 
                             max_features=20000, #в словарь попадут 20000 слов
                             analyzer='char')
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['review'], train_df['is_positive'])

eval_model(model, test_df['review'], test_df['is_positive'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Test precision = 87.48%
Test recall = 88.35%
Test f1 = 87.92%
Test accuracy = 87.86%


In [54]:
print('Positive' if test_df['is_positive'].iloc[1] else 'Negative')
eli5.show_prediction(classifier, test_df['review'].iloc[1], vec=vectorizer, 
                     targets=['positive'], target_names=['negative', 'positive'])

Positive


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## Подключаем лингвистику

### Лемматизация и стемминг

Если присмотреться, можно найти формы одного слова с разной семантической окраской по мнению классификатора. Или нет?

**Задание** Найти формы слова с разной семантической окраской.

Поверя, что они есть, попробуем что-нибудь с этим сделать.

Например, лемматизируем - сведем к начальной форме все слова. Поможет в этом библиотека spacy.

In [56]:
import spacy
from spacy import displacy

nlp = spacy.load('en_core_web_sm', disable=['parser'])

docs = [doc for doc in nlp.pipe(train_df.review.values[:50])]

In [57]:
for token in docs[0]:
    print(token.text, token.lemma_, token.ent_iob_, token.ent_type_)

Dreamgirls dreamgirl O 
, , O 
despite despite O 
its its O 
fistful fistful O 
of of O 
Tony Tony B PERSON
wins win O 
in in O 
an an B DATE
incredibly incredibly I DATE
weak weak I DATE
year year I DATE
on on O 
Broadway Broadway B FAC
, , O 
has have O 
never never O 
been be O 
what what O 
one one O 
would would O 
call call O 
a a O 
jewel jewel O 
in in O 
the the O 
crown crown O 
of of O 
stage stage O 
musicals musical O 
. . O 
However however O 
, , O 
that that O 
is be O 
not not O 
to to O 
say say O 
that that O 
in in O 
the the O 
right right O 
cinematic cinematic O 
hands hand O 
it it O 
could could O 
not not O 
be be O 
fleshed flesh O 
out out O 
and and O 
polished polish O 
into into O 
something something O 
worthwhile worthwhile O 
on on O 
- - O 
screen screen O 
. . O 
Unfortunately unfortunately O 
, , O 
what what O 
transfers transfer O 
to to O 
the the O 
screen screen O 
is be O 
basically basically O 
a a O 
slavishly slavishly O 
faithful faithful 

**Задание** Сделайте классификатор на лемматизированных текстах.

Более простой способ нормализации слов - использовать стемминг. Он немного тупой, не учитывает контекст, но иногда оказывается даже эффективнее лемматизации - а, главное, быстрее.

По сути это просто набор правил, как обрезать слово, чтобы получить основу (stem):

In [58]:
train_df.head()

,is_positive,review
0,0,"Dreamgirls, despite its fistful of Tony wins i..."
1,0,This show comes up with interesting locations ...
2,1,I simply love this movie. I also love the Ramo...
3,0,Spoilers ahead if you want to call them that.....
4,1,My all-time favorite movie! I have seen many m...


In [59]:
def lemmatize_review(text):
    doc = nlp(text)
    tokens_lemm_list = [token.lemma_ for token in doc]
    lemmatize_review = ' '.join(tokens_lemm_list)
    return lemmatize_review

In [60]:
train_df['review_lemmatized'] = train_df['review'].apply(lemmatize_review)
train_df.head()

,is_positive,review,review_lemmatized
0,0,"Dreamgirls, despite its fistful of Tony wins i...","dreamgirl , despite its fistful of Tony win in..."
1,0,This show comes up with interesting locations ...,this show come up with interesting location as...
2,1,I simply love this movie. I also love the Ramo...,I simply love this movie . I also love the Ram...
3,0,Spoilers ahead if you want to call them that.....,spoiler ahead if you want to call they that .....
4,1,My all-time favorite movie! I have seen many m...,my all - time favorite movie ! I have see many...


In [61]:
test_df.head()

,is_positive,review
0,0,A formulaic story with all the tired cliches. ...
1,1,This is both an entertaining and a touching ve...
2,0,Corey Haim is never going to be known as one o...
3,1,If you believe that any given war movie can ma...
4,1,Carla Gugino literally melts the screen in thi...


In [62]:
test_df['review_lemmatized'] = test_df['review'].apply(lemmatize_review)
test_df.head()

,is_positive,review,review_lemmatized
0,0,A formulaic story with all the tired cliches. ...,a formulaic story with all the tired cliche . ...
1,1,This is both an entertaining and a touching ve...,this be both an entertaining and a touching ve...
2,0,Corey Haim is never going to be known as one o...,Corey Haim be never go to be know as one of th...
3,1,If you believe that any given war movie can ma...,if you believe that any give war movie can mak...
4,1,Carla Gugino literally melts the screen in thi...,Carla Gugino literally melt the screen in this...


In [63]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['review_lemmatized'], train_df['is_positive'])

eval_model(model, test_df['review_lemmatized'], test_df['is_positive'])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Test precision = 88.14%
Test recall = 89.60%
Test f1 = 88.86%
Test accuracy = 88.77%


In [64]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [65]:
from string import punctuation
#знаки пунктуации
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [66]:
noise = stopwords.words("english") + list(punctuation)
print(noise)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [67]:
def lemmatize_review_clear(text):
    doc = nlp(text)
    tokens_lemm_list = [token.lemma_ for token in doc]
    tokens_lemm_list_clear = [token for token in tokens_lemm_list if not token in noise]
    lemmatize_review_clear = ' '.join(tokens_lemm_list_clear)
    return lemmatize_review_clear

In [68]:
train_df['review_lemmatized_clear'] = train_df['review'].apply(lemmatize_review_clear)
train_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear
0,0,"Dreamgirls, despite its fistful of Tony wins i...","dreamgirl , despite its fistful of Tony win in...",dreamgirl despite fistful Tony win incredibly ...
1,0,This show comes up with interesting locations ...,this show come up with interesting location as...,show come interesting location fast travel cha...
2,1,I simply love this movie. I also love the Ramo...,I simply love this movie . I also love the Ram...,I simply love movie I also love Ramones I sort...
3,0,Spoilers ahead if you want to call them that.....,spoiler ahead if you want to call they that .....,spoiler ahead want call ... I would almost r...
4,1,My all-time favorite movie! I have seen many m...,my all - time favorite movie ! I have see many...,time favorite movie I see many movie one beat ...


In [69]:
test_df['review_lemmatized_clear'] = test_df['review'].apply(lemmatize_review_clear)
test_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear
0,0,A formulaic story with all the tired cliches. ...,a formulaic story with all the tired cliche . ...,formulaic story tired cliche I shock horrible ...
1,1,This is both an entertaining and a touching ve...,this be both an entertaining and a touching ve...,entertaining touching version classic tale als...
2,0,Corey Haim is never going to be known as one o...,Corey Haim be never go to be know as one of th...,Corey Haim never go know one great actor time ...
3,1,If you believe that any given war movie can ma...,if you believe that any give war movie can mak...,believe give war movie make really feel war ne...
4,1,Carla Gugino literally melts the screen in thi...,Carla Gugino literally melt the screen in this...,Carla Gugino literally melt screen crime caper...


In [70]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['review_lemmatized_clear'], train_df['is_positive'])

eval_model(model, test_df['review_lemmatized_clear'], test_df['is_positive'])

Test precision = 87.11%
Test recall = 88.46%
Test f1 = 87.78%
Test accuracy = 87.68%


In [71]:
def lemmatize_review_entity(text):
    doc = nlp(text)
    tokens_lemm_list = [token.lemma_ if token.ent_type_ == '' else token.ent_type_ for token in doc]
    lemmatize_review_entity = ' '.join(tokens_lemm_list)
    return lemmatize_review_entity

In [72]:
train_df['review_lemmatized_entity'] = train_df['review'].apply(lemmatize_review_entity)
train_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear,review_lemmatized_entity
0,0,"Dreamgirls, despite its fistful of Tony wins i...","dreamgirl , despite its fistful of Tony win in...",dreamgirl despite fistful Tony win incredibly ...,"dreamgirl , despite its fistful of PERSON win ..."
1,0,This show comes up with interesting locations ...,this show come up with interesting location as...,show come interesting location fast travel cha...,this show come up with interesting location as...
2,1,I simply love this movie. I also love the Ramo...,I simply love this movie . I also love the Ram...,I simply love movie I also love Ramones I sort...,I simply love this movie . I also love the Ram...
3,0,Spoilers ahead if you want to call them that.....,spoiler ahead if you want to call they that .....,spoiler ahead want call ... I would almost r...,spoiler ahead if you want to call they that .....
4,1,My all-time favorite movie! I have seen many m...,my all - time favorite movie ! I have see many...,time favorite movie I see many movie one beat ...,my all - time favorite movie ! I have see many...


In [73]:
test_df['review_lemmatized_enity'] = test_df['review'].apply(lemmatize_review_entity)
test_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear,review_lemmatized_enity
0,0,A formulaic story with all the tired cliches. ...,a formulaic story with all the tired cliche . ...,formulaic story tired cliche I shock horrible ...,a formulaic story with all the tired cliche . ...
1,1,This is both an entertaining and a touching ve...,this be both an entertaining and a touching ve...,entertaining touching version classic tale als...,this be both an entertaining and a touching ve...
2,0,Corey Haim is never going to be known as one o...,Corey Haim be never go to be know as one of th...,Corey Haim never go know one great actor time ...,PERSON PERSON be never go to be know as one of...
3,1,If you believe that any given war movie can ma...,if you believe that any give war movie can mak...,believe give war movie make really feel war ne...,if you believe that any give war movie can mak...
4,1,Carla Gugino literally melts the screen in thi...,Carla Gugino literally melt the screen in this...,Carla Gugino literally melt screen crime caper...,PERSON PERSON literally melt the screen in thi...


In [74]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['review_lemmatized_entity'], train_df['is_positive'])

eval_model(model, test_df['review_lemmatized_enity'], test_df['is_positive'])

Test precision = 87.74%
Test recall = 89.95%
Test f1 = 88.83%
Test accuracy = 88.69%


In [75]:
def lemmatize_review_entity_clear(text):
    doc = nlp(text)
    tokens_lemm_list = [token.lemma_ if token.ent_type_ == '' else token.ent_type_ for token in doc]
    tokens_lemm_list_clear = [token for token in tokens_lemm_list if not token in noise]
    lemmatize_review_entity_clear = ' '.join(tokens_lemm_list_clear)
    return lemmatize_review_entity_clear 

In [76]:
train_df['review_lemmatized_entity_clear'] = train_df['review'].apply(lemmatize_review_entity_clear)
train_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear,review_lemmatized_entity,review_lemmatized_entity_clear
0,0,"Dreamgirls, despite its fistful of Tony wins i...","dreamgirl , despite its fistful of Tony win in...",dreamgirl despite fistful Tony win incredibly ...,"dreamgirl , despite its fistful of PERSON win ...",dreamgirl despite fistful PERSON win DATE DATE...
1,0,This show comes up with interesting locations ...,this show come up with interesting location as...,show come interesting location fast travel cha...,this show come up with interesting location as...,show come interesting location fast travel cha...
2,1,I simply love this movie. I also love the Ramo...,I simply love this movie . I also love the Ram...,I simply love movie I also love Ramones I sort...,I simply love this movie . I also love the Ram...,I simply love movie I also love Ramones I sort...
3,0,Spoilers ahead if you want to call them that.....,spoiler ahead if you want to call they that .....,spoiler ahead want call ... I would almost r...,spoiler ahead if you want to call they that .....,spoiler ahead want call ... I would almost r...
4,1,My all-time favorite movie! I have seen many m...,my all - time favorite movie ! I have see many...,time favorite movie I see many movie one beat ...,my all - time favorite movie ! I have see many...,time favorite movie I see many movie one beat ...


In [77]:
test_df['review_lemmatized_enity_Clear'] = test_df['review'].apply(lemmatize_review_entity_clear)
test_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear,review_lemmatized_enity,review_lemmatized_enity_Clear
0,0,A formulaic story with all the tired cliches. ...,a formulaic story with all the tired cliche . ...,formulaic story tired cliche I shock horrible ...,a formulaic story with all the tired cliche . ...,formulaic story tired cliche I shock horrible ...
1,1,This is both an entertaining and a touching ve...,this be both an entertaining and a touching ve...,entertaining touching version classic tale als...,this be both an entertaining and a touching ve...,entertaining touching version classic tale als...
2,0,Corey Haim is never going to be known as one o...,Corey Haim be never go to be know as one of th...,Corey Haim never go know one great actor time ...,PERSON PERSON be never go to be know as one of...,PERSON PERSON never go know one great actor ti...
3,1,If you believe that any given war movie can ma...,if you believe that any give war movie can mak...,believe give war movie make really feel war ne...,if you believe that any give war movie can mak...,believe give war movie make really feel war ne...
4,1,Carla Gugino literally melts the screen in thi...,Carla Gugino literally melt the screen in this...,Carla Gugino literally melt screen crime caper...,PERSON PERSON literally melt the screen in thi...,PERSON PERSON literally melt screen crime cape...


In [78]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['review_lemmatized_entity_clear'], train_df['is_positive'])

eval_model(model, test_df['review_lemmatized_enity_Clear'], test_df['is_positive'])

Test precision = 85.92%
Test recall = 88.90%
Test f1 = 87.38%
Test accuracy = 87.16%


In [79]:
from nltk import PorterStemmer

stemmer = PorterStemmer()

print(stemmer.stem('become'))
print(stemmer.stem('becomes'))
print(stemmer.stem('became'))

becom
becom
becam


**Задание** Попробуйте вместо лемм классифицировать основы.

In [80]:
def stem_review(text):
    list_word = text.split()
    list_stem = [stemmer.stem(word) for word in list_word]
    stem_text = ' '.join(list_stem)
    return stem_text

In [81]:
train_df['stem_review'] = train_df['review'].apply(stem_review)
train_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear,review_lemmatized_entity,review_lemmatized_entity_clear,stem_review
0,0,"Dreamgirls, despite its fistful of Tony wins i...","dreamgirl , despite its fistful of Tony win in...",dreamgirl despite fistful Tony win incredibly ...,"dreamgirl , despite its fistful of PERSON win ...",dreamgirl despite fistful PERSON win DATE DATE...,"dreamgirls, despit it fist of toni win in an i..."
1,0,This show comes up with interesting locations ...,this show come up with interesting location as...,show come interesting location fast travel cha...,this show come up with interesting location as...,show come interesting location fast travel cha...,thi show come up with interest locat as fast a...
2,1,I simply love this movie. I also love the Ramo...,I simply love this movie . I also love the Ram...,I simply love movie I also love Ramones I sort...,I simply love this movie . I also love the Ram...,I simply love movie I also love Ramones I sort...,i simpli love thi movie. i also love the ramon...
3,0,Spoilers ahead if you want to call them that.....,spoiler ahead if you want to call they that .....,spoiler ahead want call ... I would almost r...,spoiler ahead if you want to call they that .....,spoiler ahead want call ... I would almost r...,spoiler ahead if you want to call them that......
4,1,My all-time favorite movie! I have seen many m...,my all - time favorite movie ! I have see many...,time favorite movie I see many movie one beat ...,my all - time favorite movie ! I have see many...,time favorite movie I see many movie one beat ...,my all-tim favorit movie! i have seen mani mov...


In [82]:
test_df['stem_review'] = test_df['review'].apply(stem_review)
test_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear,review_lemmatized_enity,review_lemmatized_enity_Clear,stem_review
0,0,A formulaic story with all the tired cliches. ...,a formulaic story with all the tired cliche . ...,formulaic story tired cliche I shock horrible ...,a formulaic story with all the tired cliche . ...,formulaic story tired cliche I shock horrible ...,a formula stori with all the tire cliches. i w...
1,1,This is both an entertaining and a touching ve...,this be both an entertaining and a touching ve...,entertaining touching version classic tale als...,this be both an entertaining and a touching ve...,entertaining touching version classic tale als...,thi is both an entertain and a touch version o...
2,0,Corey Haim is never going to be known as one o...,Corey Haim be never go to be know as one of th...,Corey Haim never go know one great actor time ...,PERSON PERSON be never go to be know as one of...,PERSON PERSON never go know one great actor ti...,corey haim is never go to be known as one of t...
3,1,If you believe that any given war movie can ma...,if you believe that any give war movie can mak...,believe give war movie make really feel war ne...,if you believe that any give war movie can mak...,believe give war movie make really feel war ne...,if you believ that ani given war movi can make...
4,1,Carla Gugino literally melts the screen in thi...,Carla Gugino literally melt the screen in this...,Carla Gugino literally melt screen crime caper...,PERSON PERSON literally melt the screen in thi...,PERSON PERSON literally melt screen crime cape...,carla gugino liter melt the screen in thi crim...


In [83]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['stem_review'], train_df['is_positive'])

eval_model(model, test_df['stem_review'], test_df['is_positive'])

Test precision = 87.70%
Test recall = 89.06%
Test f1 = 88.37%
Test accuracy = 88.28%


In [84]:
def stem_review_clear(text):
    list_word = text.split()
    list_stem = [stemmer.stem(word) for word in list_word]
    list_stem_clear = [stem for stem in list_stem if not stem in noise]
    stem_text_clear = ' '.join(list_stem_clear)
    return stem_text_clear

In [85]:
train_df['stem_review_clear'] = train_df['review'].apply(stem_review_clear)
train_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear,review_lemmatized_entity,review_lemmatized_entity_clear,stem_review,stem_review_clear
0,0,"Dreamgirls, despite its fistful of Tony wins i...","dreamgirl , despite its fistful of Tony win in...",dreamgirl despite fistful Tony win incredibly ...,"dreamgirl , despite its fistful of PERSON win ...",dreamgirl despite fistful PERSON win DATE DATE...,"dreamgirls, despit it fist of toni win in an i...","dreamgirls, despit fist toni win incred weak y..."
1,0,This show comes up with interesting locations ...,this show come up with interesting location as...,show come interesting location fast travel cha...,this show come up with interesting location as...,show come interesting location fast travel cha...,thi show come up with interest locat as fast a...,thi show come interest locat fast travel chann...
2,1,I simply love this movie. I also love the Ramo...,I simply love this movie . I also love the Ram...,I simply love movie I also love Ramones I sort...,I simply love this movie . I also love the Ram...,I simply love movie I also love Ramones I sort...,i simpli love thi movie. i also love the ramon...,"simpli love thi movie. also love ramones, sort..."
3,0,Spoilers ahead if you want to call them that.....,spoiler ahead if you want to call they that .....,spoiler ahead want call ... I would almost r...,spoiler ahead if you want to call they that .....,spoiler ahead want call ... I would almost r...,spoiler ahead if you want to call them that......,spoiler ahead want call that... would almost r...
4,1,My all-time favorite movie! I have seen many m...,my all - time favorite movie ! I have see many...,time favorite movie I see many movie one beat ...,my all - time favorite movie ! I have see many...,time favorite movie I see many movie one beat ...,my all-tim favorit movie! i have seen mani mov...,"all-tim favorit movie! seen mani movies, thi o..."


In [86]:
test_df['stem_review_clear'] = test_df['review'].apply(stem_review_clear)
test_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear,review_lemmatized_enity,review_lemmatized_enity_Clear,stem_review,stem_review_clear
0,0,A formulaic story with all the tired cliches. ...,a formulaic story with all the tired cliche . ...,formulaic story tired cliche I shock horrible ...,a formulaic story with all the tired cliche . ...,formulaic story tired cliche I shock horrible ...,a formula stori with all the tire cliches. i w...,formula stori tire cliches. wa shock horribl s...
1,1,This is both an entertaining and a touching ve...,this be both an entertaining and a touching ve...,entertaining touching version classic tale als...,this be both an entertaining and a touching ve...,entertaining touching version classic tale als...,thi is both an entertain and a touch version o...,"thi entertain touch version classic tale, also..."
2,0,Corey Haim is never going to be known as one o...,Corey Haim be never go to be know as one of th...,Corey Haim never go know one great actor time ...,PERSON PERSON be never go to be know as one of...,PERSON PERSON never go know one great actor ti...,corey haim is never go to be known as one of t...,corey haim never go known one great actor hi t...
3,1,If you believe that any given war movie can ma...,if you believe that any give war movie can mak...,believe give war movie make really feel war ne...,if you believe that any give war movie can mak...,believe give war movie make really feel war ne...,if you believ that ani given war movi can make...,believ ani given war movi make realli feel war...
4,1,Carla Gugino literally melts the screen in thi...,Carla Gugino literally melt the screen in this...,Carla Gugino literally melt screen crime caper...,PERSON PERSON literally melt the screen in thi...,PERSON PERSON literally melt screen crime cape...,carla gugino liter melt the screen in thi crim...,carla gugino liter melt screen thi crime caper...


In [87]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['stem_review_clear'], train_df['is_positive'])

eval_model(model, test_df['stem_review_clear'], test_df['is_positive'])

Test precision = 87.48%
Test recall = 88.47%
Test f1 = 87.97%
Test accuracy = 87.90%


### NER

В текстах рецензий очень много именованных сущностей. Вот, например:

In [88]:
displacy.render(docs[0], style='ent', jupyter=True)

Вообще говоря, почему вдруг какой-нибудь Депп должен нести семантическую окраску? Однако оказывается, что классификатор выучивает, что какие-то имена чаще в положительных рецензиях - или наоборот. Это похоже на переобучение - почему бы не попробовать вырезать сущности?

**Задание** Удалите из текстов какие-то из сущностей, пользуясь координатами из запикленных файлов. Описание сущностей можно посмотреть [здесь](https://spacy.io/api/annotation#named-entities). Запустите классификатор.

In [89]:
def del_entity_ORG(text):
    doc = nlp(text)
    tokens_lemm_list = [token.lemma_ for token in doc if token.ent_type_ !='ORG']
    del_entity_ORG  = ' '.join(tokens_lemm_list)
    return del_entity_ORG

In [90]:
train_df['lemm_del_entity_ORG'] = train_df['review'].apply(del_entity_ORG)
train_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear,review_lemmatized_entity,review_lemmatized_entity_clear,stem_review,stem_review_clear,lemm_del_entity_ORG
0,0,"Dreamgirls, despite its fistful of Tony wins i...","dreamgirl , despite its fistful of Tony win in...",dreamgirl despite fistful Tony win incredibly ...,"dreamgirl , despite its fistful of PERSON win ...",dreamgirl despite fistful PERSON win DATE DATE...,"dreamgirls, despit it fist of toni win in an i...","dreamgirls, despit fist toni win incred weak y...","dreamgirl , despite its fistful of Tony win in..."
1,0,This show comes up with interesting locations ...,this show come up with interesting location as...,show come interesting location fast travel cha...,this show come up with interesting location as...,show come interesting location fast travel cha...,thi show come up with interest locat as fast a...,thi show come interest locat fast travel chann...,this show come up with interesting location as...
2,1,I simply love this movie. I also love the Ramo...,I simply love this movie . I also love the Ram...,I simply love movie I also love Ramones I sort...,I simply love this movie . I also love the Ram...,I simply love movie I also love Ramones I sort...,i simpli love thi movie. i also love the ramon...,"simpli love thi movie. also love ramones, sort...",I simply love this movie . I also love the Ram...
3,0,Spoilers ahead if you want to call them that.....,spoiler ahead if you want to call they that .....,spoiler ahead want call ... I would almost r...,spoiler ahead if you want to call they that .....,spoiler ahead want call ... I would almost r...,spoiler ahead if you want to call them that......,spoiler ahead want call that... would almost r...,spoiler ahead if you want to call they that .....
4,1,My all-time favorite movie! I have seen many m...,my all - time favorite movie ! I have see many...,time favorite movie I see many movie one beat ...,my all - time favorite movie ! I have see many...,time favorite movie I see many movie one beat ...,my all-tim favorit movie! i have seen mani mov...,"all-tim favorit movie! seen mani movies, thi o...",my all - time favorite movie ! I have see many...


In [91]:
test_df['lemm_del_entity_ORG'] = test_df['review'].apply(del_entity_ORG)
test_df.head()

,is_positive,review,review_lemmatized,review_lemmatized_clear,review_lemmatized_enity,review_lemmatized_enity_Clear,stem_review,stem_review_clear,lemm_del_entity_ORG
0,0,A formulaic story with all the tired cliches. ...,a formulaic story with all the tired cliche . ...,formulaic story tired cliche I shock horrible ...,a formulaic story with all the tired cliche . ...,formulaic story tired cliche I shock horrible ...,a formula stori with all the tire cliches. i w...,formula stori tire cliches. wa shock horribl s...,a formulaic story with all the tired cliche . ...
1,1,This is both an entertaining and a touching ve...,this be both an entertaining and a touching ve...,entertaining touching version classic tale als...,this be both an entertaining and a touching ve...,entertaining touching version classic tale als...,thi is both an entertain and a touch version o...,"thi entertain touch version classic tale, also...",this be both an entertaining and a touching ve...
2,0,Corey Haim is never going to be known as one o...,Corey Haim be never go to be know as one of th...,Corey Haim never go know one great actor time ...,PERSON PERSON be never go to be know as one of...,PERSON PERSON never go know one great actor ti...,corey haim is never go to be known as one of t...,corey haim never go known one great actor hi t...,Corey Haim be never go to be know as one of th...
3,1,If you believe that any given war movie can ma...,if you believe that any give war movie can mak...,believe give war movie make really feel war ne...,if you believe that any give war movie can mak...,believe give war movie make really feel war ne...,if you believ that ani given war movi can make...,believ ani given war movi make realli feel war...,if you believe that any give war movie can mak...
4,1,Carla Gugino literally melts the screen in thi...,Carla Gugino literally melt the screen in this...,Carla Gugino literally melt screen crime caper...,PERSON PERSON literally melt the screen in thi...,PERSON PERSON literally melt screen crime cape...,carla gugino liter melt the screen in thi crim...,carla gugino liter melt screen thi crime caper...,Carla Gugino literally melt the screen in this...


In [115]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
classifier = LogisticRegression()

model = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', classifier)
])

model.fit(train_df['lemm_del_entity_ORG'], train_df['is_positive'])

eval_model(model, test_df['lemm_del_entity_ORG'], test_df['is_positive'])

Test precision = 88.19%
Test recall = 89.64%
Test f1 = 88.91%
Test accuracy = 88.82%


## Включаем дип лёрнинг

Мы тут пришли deep learning'ом заниматься, а делаем почему-то модель на логистической регрессии. Как так?

Попробуем запустить относительно стандартную модель для классификации текстов - сверточная сеть поверх словных эмбеддингов.

Разбираться, что это за зверь, будем на следующих занятиях, а пока будем просто им пользоваться :)

Каждое предложение нужно представлять набором слов - и сразу же начинаются проблемы. Во-первых, как ограничить длину предложения?

Прикинем по гистограмме, какая длина нам подходит:

In [ ]:
train_df['review']

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

_, _, hist = plt.hist(train_df.review.apply(lambda text: len(text.split())), bins='auto')
hist

Кроме этого, нужно перенумеровать как-то слова.

In [ ]:
from collections import Counter

words_counter = Counter((word for text in train_df.review for word in text.lower().split()))

word2idx = {
    '': 0,
    '<unk>': 1
}
for word, count in words_counter.most_common():
    if count < 10:
        break
        
    word2idx[word] = len(word2idx)
    
print('Words count', len(word2idx))

**Задание** Сконвертируйте данные

In [ ]:
def convert(texts, word2idx, max_text_len):
    data = np.zeros((len(texts), max_text_len), dtype=np.int)
    
    for inx, text in enumerate(texts):
        result = []
        for word in text.split():
            if word in word2idx:
                result.append(word2idx[word])
        padding = [0]*(max_text_len - len(result))
        data[inx] = np.array(padding + result[-max_text_len:], dtype=np.int)
    return data

X_train = convert(train_df.review, word2idx, 1000)
X_test = convert(test_df.review, word2idx, 1000)

Поставим учиться модельку на keras.

*Напоминание*: на keras, чтобы обучить модель, нужно
1. Определить модель, например:
```python 
model = Sequential()
model.add(Dense(1, activation='sigmoid', input_dim=NUM_WORDS))
```
2. Задать функцию потерь и оптимизатор:
```python
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
```

3. Запустить обучение:
```python
model.fit(X_train, y_train, 
          batch_size=32,
          epochs=3,
          validation_data=(X_test, y_test))
```

В NLP чаще всего ставятся задачи классификации, поэтому нужно запомнить такие функции потерь:

*   **categorical_crossentropy** - для многоклассовой классификации, в качестве меток должны передаваться one-hot-encoding вектора
*   **sparse_categorical_crossentropy** - аналогично предыдущему, но в качестве меток нужно передавать просто индексы соответствующих классов
*   **binary_crossentropy** - для бинарной классификации


В качестве оптимизатора обычно используют `sgd` или `adam`.


In [107]:
import tensorflow as tf
tf.__version__

'2.8.2'

In [108]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout

In [109]:
model = Sequential([
    Embedding(input_dim=len(word2idx), output_dim=64, input_shape=(X_train.shape[1],)),
    GlobalMaxPooling1D(),
    Dense(units=10, activation='relu'),
    Dense(units=10, activation='relu'),

    Dense(units=1, activation='sigmoid')
])

model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1000, 64)          1714112   
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 1,714,883
Trainable params: 1,714,883
Non-trainable params: 0
____________________________________________

In [110]:
model.fit(X_train, train_df.is_positive, batch_size=128, epochs=10, 
          validation_data=(X_test, test_df.is_positive))

Epoch 1/10
196/196 [==============================] - 3s 13ms/step - loss: 0.6163 - accuracy: 0.7437 - val_loss: 0.4241 - val_accuracy: 0.8369
Epoch 2/10
196/196 [==============================] - 2s 10ms/step - loss: 0.3092 - accuracy: 0.8776 - val_loss: 0.2994 - val_accuracy: 0.8724
Epoch 3/10
196/196 [==============================] - 2s 10ms/step - loss: 0.1891 - accuracy: 0.9287 - val_loss: 0.2881 - val_accuracy: 0.8796
Epoch 4/10
196/196 [==============================] - 2s 10ms/step - loss: 0.1108 - accuracy: 0.9638 - val_loss: 0.3074 - val_accuracy: 0.8777
Epoch 5/10
196/196 [==============================] - 2s 10ms/step - loss: 0.0563 - accuracy: 0.9865 - val_loss: 0.3415 - val_accuracy: 0.8749
Epoch 6/10
196/196 [==============================] - 2s 10ms/step - loss: 0.0252 - accuracy: 0.9963 - val_loss: 0.3827 - val_accuracy: 0.8719
Epoch 7/10
196/196 [==============================] - 2s 10ms/step - loss: 0.0108 - accuracy: 0.9992 - val_loss: 0.4169 - val_accuracy: 0.8719

**Задание** Подсчитайте качество модели на тесте

In [111]:
preds = model.predict(X_test)
preds

array([[6.9768197e-04],
       [9.9998474e-01],
       [3.6493152e-01],
       ...,
       [9.9535215e-01],
       [9.9982566e-01],
       [9.9545789e-01]], dtype=float32)

In [112]:
threshold = 0.5
preds[preds >= threshold] = 1
preds[preds < threshold] = 0
preds.T[0]

array([0., 1., 0., ..., 1., 1., 1.], dtype=float32)

In [113]:
test_df.is_positive.to_numpy()

array([0, 1, 0, ..., 1, 1, 1])

In [114]:
accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(preds, test_df.is_positive.values)
accuracy.result().numpy()

0.87056